# Faact: Geo-Unblocking Infrastructure Deployment

This notebook provides the code and instructions to deploy a network for accessing geo-restricted content. The architecture uses a Google Cloud Platform (GCP) instance as an exit node and a transit server in Hong Kong to forward traffic.

### Architecture

The traffic flow is as follows:

**`Your Computer`** --- (Hysteria Proxy) ---> **`Hong Kong VPS`** --- (WireGuard Tunnel) ---> **`GCP Server (US)`** --- > **`Internet`**

**Components:**
- **Terraform:** To automatically provision the required GCP resources.
- **WireGuard:** To create a secure VPN tunnel between the GCP server and the Hong Kong VPS.
- **Hysteria:** A high-performance proxy to route your local traffic to the Hong Kong VPS.

### Prerequisites

1.  **GCP Account:** A Google Cloud project with billing enabled.
2.  **Hong Kong VPS:** A virtual private server from any provider, located in Hong Kong.
3.  **Local Tools:** 
    - `terraform`
    - `gcloud` CLI (authenticated with your GCP account)
    - `wireguard-tools` (for key generation, optional)
    - A Hysteria client binary

## Part 1: Provision GCP Infrastructure with Terraform

This cell writes the `main.tf` file to your local disk. It defines the VPC, subnet, firewall rules, and the GCE instance on Google Cloud.

In [ ]:
%%writefile main.tf

terraform {
  required_providers {
    google = {
      source  = "hashicorp/google"
      version = ">= 4.50.0"
    }
  }
}

provider "google" {
  project = "your-gcp-project-id" # <<< REPLACE WITH YOUR GCP PROJECT ID
  region  = "us-west1"
}

resource "google_compute_network" "tiktok_stream_vpc" {
  name                    = "tiktok-stream-vpc"
  auto_create_subnetworks = false
}

resource "google_compute_subnetwork" "tiktok_stream_subnet" {
  name          = "tiktok-stream-subnet-us-west1"
  ip_cidr_range = "10.10.1.0/24"
  region        = "us-west1"
  network       = google_compute_network.tiktok_stream_vpc.id
}

resource "google_compute_firewall" "allow_stream_traffic" {
  name    = "allow-stream-traffic"
  network = google_compute_network.tiktok_stream_vpc.name

  allow {
    protocol = "udp"
    ports    = ["51820"] # WireGuard port
  }

  allow {
    protocol = "tcp"
    ports    = ["22"] # SSH port
  }

  source_ranges = ["0.0.0.0/0"]
}

resource "google_compute_instance" "wg_server_usw1" {
  name         = "wg-server-us-west1"
  machine_type = "e2-small"
  zone         = "us-west1-b"

  tags = ["wireguard-server"]

  boot_disk {
    initialize_params {
      image = "debian-cloud/debian-11"
    }
  }

  network_interface {
    subnetwork   = google_compute_subnetwork.tiktok_stream_subnet.id
    network_ip   = "10.10.1.10"
    network_tier = "PREMIUM"
    access_config {
      // Assign temporary public IP
    }
  }

  metadata_startup_script = <<-EOT
    #! /bin/bash
    apt-get update
    apt-get install -y wireguard iptables
    echo "net.ipv4.ip_forward=1" >> /etc/sysctl.conf
    sysctl -p
  EOT

  service_account {
    scopes = ["https://www.googleapis.com/auth/cloud-platform"]
  }
}

output "wg_server_ip" {
  value = google_compute_instance.wg_server_usw1.network_interface[0].access_config[0].nat_ip
}

Now, run the following cells to apply the Terraform configuration. Ensure you have authenticated with `gcloud` first.

In [ ]:
!terraform init

In [ ]:
!terraform apply --auto-approve

Capture the public IP address of your new GCP server. You will need this for the next steps.

In [ ]:
!terraform output wg_server_ip

## Part 2: Configure the GCP & Hong Kong Servers

You now need to configure WireGuard on both servers and Hysteria on the Hong Kong server. This involves generating key pairs and creating configuration files.

### Step 2.1: Generate Keys

SSH into **both** your GCP instance and your Hong Kong VPS and run the following command on each to generate a private and public key pair. Keep the terminal windows open.

**Run on both servers:**

In [ ]:
!umask 077; wg genkey | tee privatekey | wg pubkey > publickey

### Step 2.2: Create Configuration Files

You will now create the configuration files. **You must swap the public keys between the servers.**

- `<GCP_SERVER_PUBLIC_IP>`: The IP address from the `terraform output` command.
- `<GCP_SERVER_PRIVATE_KEY>`: The content of the `privatekey` file on the **GCP** server.
- `<HK_CLIENT_PUBLIC_KEY>`: The content of the `publickey` file on the **Hong Kong** server.
- `<HK_CLIENT_PRIVATE_KEY>`: The content of the `privatekey` file on the **Hong Kong** server.
- `<GCP_SERVER_PUBLIC_KEY>`: The content of the `publickey` file on the **GCP** server.
- `YOUR_VERY_SECRET_PASSWORD`: Create a strong password for Hysteria.

#### On the GCP Server

Create `/etc/wireguard/wg0.conf` with the following content:

In [ ]:
%%writefile gcp_wg0.conf

[Interface]
Address = 192.168.10.1/24
ListenPort = 51820
PrivateKey = <GCP_SERVER_PRIVATE_KEY>
PostUp = iptables -A FORWARD -i %i -j ACCEPT; iptables -t nat -A POSTROUTING -o eth0 -j MASQUERADE
PostDown = iptables -D FORWARD -i %i -j ACCEPT; iptables -t nat -D POSTROUTING -o eth0 -j MASQUERADE

[Peer]
PublicKey = <HK_CLIENT_PUBLIC_KEY>
AllowedIPs = 192.168.10.2/32

#### On the Hong Kong Server

Create `/etc/wireguard/wg0.conf` with the following content:

In [ ]:
%%writefile hk_wg0.conf

[Interface]
Address = 192.168.10.2/24
PrivateKey = <HK_CLIENT_PRIVATE_KEY>

[Peer]
PublicKey = <GCP_SERVER_PUBLIC_KEY>
Endpoint = <GCP_SERVER_PUBLIC_IP>:51820
AllowedIPs = 0.0.0.0/0, ::/0
PersistentKeepalive = 25

Also on the **Hong Kong** server, create `/etc/hysteria/config.yaml`:

In [ ]:
%%writefile hk_hysteria_config.yaml

listen: :443

auth:
  type: password
  password: "YOUR_VERY_SECRET_PASSWORD"

transport:
  type: udp
  udp:
    rate: 50 mbps
    recvRate: 100 mbps

### Step 2.3: Start Services

**On both servers**, start the WireGuard interface:
```bash
wg-quick up wg0
```

**On the Hong Kong server**, start Hysteria (you may need to adapt this command depending on how you installed it):
```bash
hysteria server -c /etc/hysteria/config.yaml
```

## Part 3: Configure and Run the Local Client

Now, create the client configuration file on your local machine. This tells your Hysteria client how to connect to the Hong Kong VPS.

In [ ]:
%%writefile client-config.yaml

server: <HK_VPS_PUBLIC_IP>:443
auth:
  type: password
  password: "YOUR_VERY_SECRET_PASSWORD"

transport:
  type: udp
  udp:
    rate: 50 mbps
    recvRate: 100 mbps

socks5:
  listen: 127.0.0.1:1080

Run the Hysteria client locally. This will start a SOCKS5 proxy on port `1080`.

In [ ]:
!./hysteria-linux-amd64-avx client -c client-config.yaml

## Part 4: Testing

1.  Configure your web browser or application to use the SOCKS5 proxy at `127.0.0.1:1080`.
2.  Visit a site like `ifconfig.me` to verify that your public IP address matches the GCP server's IP.
3.  You should now be able to access the desired content.

## Part 5: Cleanup

To avoid ongoing charges, destroy the GCP infrastructure when you are finished.

In [ ]:
!terraform destroy --auto-approve